In [5]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

In [6]:

import datetime
import time
import requests
from urllib.request import urlopen
import json
import pandas as pd
import folium
from src.functions import *
import warnings
warnings.filterwarnings("ignore")

Representing the Target cities

In [7]:
target_url='https://repo.hops.works/dev/jdowling/target_cities.json'
response = urlopen(target_url)
target_cities = json.loads(response.read())

In [8]:
target_cities

{'EU': {'Amsterdam': [52.37, 4.89],
  'Athina': [37.98, 23.73],
  'Berlin': [52.52, 13.39],
  'Gdansk': [54.37, 18.61],
  'Kraków': [50.06, 19.94],
  'London': [51.51, -0.13],
  'Madrid': [40.42, -3.7],
  'Marseille': [43.3, 5.37],
  'Milano': [45.46, 9.19],
  'München': [48.14, 11.58],
  'Napoli': [40.84, 14.25],
  'Paris': [48.85, 2.35],
  'Sevilla': [37.39, -6.0],
  'Stockholm': [59.33, 18.07],
  'Tallinn': [59.44, 24.75],
  'Varna': [43.21, 27.92],
  'Wien': [48.21, 16.37]},
 'US': {'Albuquerque': [35.08, -106.65],
  'Atlanta': [33.75, -84.39],
  'Chicago': [41.88, -87.62],
  'Columbus': [39.96, -83.0],
  'Dallas': [32.78, -96.8],
  'Denver': [39.74, -104.98],
  'Houston': [29.76, -95.37],
  'Los Angeles': [34.05, -118.24],
  'New York': [40.71, -74.01],
  'Phoenix-Mesa': [33.66, -112.04],
  'Salt Lake City': [40.76, -111.89],
  'San Francisco': [37.78, -122.42],
  'Tampa': [27.95, -82.46]},
 'Seattle': {'Bellevue-SE 12th St': [47.60086, -122.1484],
  'DARRINGTON - FIR ST (Darringt

European Environmental Agency

In [10]:
target_cities["EU"]

{'Amsterdam': [52.37, 4.89],
 'Athina': [37.98, 23.73],
 'Berlin': [52.52, 13.39],
 'Gdansk': [54.37, 18.61],
 'Kraków': [50.06, 19.94],
 'London': [51.51, -0.13],
 'Madrid': [40.42, -3.7],
 'Marseille': [43.3, 5.37],
 'Milano': [45.46, 9.19],
 'München': [48.14, 11.58],
 'Napoli': [40.84, 14.25],
 'Paris': [48.85, 2.35],
 'Sevilla': [37.39, -6.0],
 'Stockholm': [59.33, 18.07],
 'Tallinn': [59.44, 24.75],
 'Varna': [43.21, 27.92],
 'Wien': [48.21, 16.37]}

In [15]:

df_eu = pd.read_csv("../data/backfill_pm2_5_eu.csv")
df_eu.isna().sum().sum()
print("Size of this dataframe:", df_eu.shape)

df_eu.sample(3)

Size of this dataframe: (63548, 3)


,city_name,date,pm2_5
11980,Gdansk,2014-12-23,8.0
62593,Wien,2020-08-30,7.0
32986,Milano,2021-02-11,17.0


United States Environmental Protection Agency

In [16]:
target_cities["US"]

{'Albuquerque': [35.08, -106.65],
 'Atlanta': [33.75, -84.39],
 'Chicago': [41.88, -87.62],
 'Columbus': [39.96, -83.0],
 'Dallas': [32.78, -96.8],
 'Denver': [39.74, -104.98],
 'Houston': [29.76, -95.37],
 'Los Angeles': [34.05, -118.24],
 'New York': [40.71, -74.01],
 'Phoenix-Mesa': [33.66, -112.04],
 'Salt Lake City': [40.76, -111.89],
 'San Francisco': [37.78, -122.42],
 'Tampa': [27.95, -82.46]}

In [18]:
df_us = pd.read_csv("../data/backfill_pm2_5_us.csv")

In [19]:
df_us.isna().sum().sum()

0

In [20]:
print("Size of this dataframe:", df_us.shape)

df_us.sample(3)

Size of this dataframe: (46037, 3)


,date,city_name,pm2_5
10489,2021-10-02,Chicago,7.4
8879,2017-05-06,Chicago,3.1
16990,2022-09-30,Dallas,11.4


Processing special city - Seattle


In [21]:
target_cities["Seattle"]

{'Bellevue-SE 12th St': [47.60086, -122.1484],
 'DARRINGTON - FIR ST (Darrington High School)': [48.2469, -121.6031],
 'KENT - JAMES & CENTRAL': [47.38611, -122.23028],
 'LAKE FOREST PARK TOWNE CENTER': [47.755, -122.2806],
 'MARYSVILLE - 7TH AVE (Marysville Junior High)': [48.05432, -122.17153],
 'NORTH BEND - NORTH BEND WAY': [47.49022, -121.77278],
 'SEATTLE - BEACON HILL': [47.56824, -122.30863],
 'SEATTLE - DUWAMISH': [47.55975, -122.33827],
 'SEATTLE - SOUTH PARK #2': [47.53091, -122.3208],
 'Seattle-10th & Weller': [47.59722, -122.31972],
 'TACOMA - ALEXANDER AVE': [47.2656, -122.3858],
 'TACOMA - L STREET': [47.1864, -122.4517],
 'Tacoma-S 36th St': [47.22634, -122.46256],
 'Tukwila Allentown': [47.49854, -122.27839],
 'Tulalip-Totem Beach Rd': [48.06534, -122.28519]}

In [22]:
df_seattle = pd.read_csv("../data/backfill_pm2_5_seattle.csv")

In [23]:
print("Size of this dataframe:", df_seattle.shape)

df_seattle.sample(3)

Size of this dataframe: (46479, 3)


,city_name,date,pm2_5
41261,SEATTLE - DUWAMISH,2022-03-28,5.8
41083,Bellevue-SE 12th St,2022-10-20,101.4
8606,SEATTLE - DUWAMISH,2015-12-31,11.5


In [24]:
df_seattle.city_name.value_counts()

city_name
NORTH BEND - NORTH BEND WAY                      3705
TACOMA - L STREET                                3696
SEATTLE - BEACON HILL                            3691
MARYSVILLE - 7TH AVE (Marysville Junior High)    3648
DARRINGTON - FIR ST (Darrington High School)     3614
SEATTLE - SOUTH PARK #2                          3577
TACOMA - ALEXANDER AVE                           3569
KENT - JAMES & CENTRAL                           3556
SEATTLE - DUWAMISH                               3439
Seattle-10th & Weller                            3097
LAKE FOREST PARK TOWNE CENTER                    2999
Tacoma-S 36th St                                 2574
Bellevue-SE 12th St                              2172
Tukwila Allentown                                2074
Tulalip-Totem Beach Rd                           1068
Name: count, dtype: int64

All together

In [25]:
df_air_quality = pd.concat([df_eu, df_us, df_seattle]).reset_index(drop=True)

In [26]:
df_air_quality.sample(5)

,city_name,date,pm2_5
19764,London,2015-09-27,6.0
134361,Seattle-10th & Weller,2019-01-01,16.5
81919,Denver,2016-07-14,7.7
137008,Tacoma-S 36th St,2019-06-12,7.7
11536,Gdansk,2013-10-05,11.0


Loading Weather Data from Open Meteo

In [27]:

df_weather = pd.read_csv("../data/backfill_weather.csv")
df_weather.city_name.value_counts()
df_weather.sample(3)


,city_name,date,temperature_max,temperature_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_max,wind_gusts_max,wind_direction_dominant
5667,Athina,2018-03-23,16.6,10.1,0.7,0.7,0.0,3.0,22.7,63.7,225
95665,New York,2017-07-24,23.9,19.2,46.7,46.7,0.0,13.0,24.3,40.7,60
158017,Tacoma-S 36th St,2013-04-08,12.2,4.2,1.6,1.6,0.0,4.0,13.6,34.9,205


In [28]:
df_air_quality.date = pd.to_datetime(df_air_quality.date)
df_weather.date = pd.to_datetime(df_weather.date)

df_air_quality["unix_time"] = df_air_quality["date"].apply(convert_date_to_unix)
df_weather["unix_time"] = df_weather["date"].apply(convert_date_to_unix)

In [29]:
df_air_quality.date = df_air_quality.date.astype(str)
df_weather.date = df_weather.date.astype(str)

In [30]:
df_air_quality

,city_name,date,pm2_5,unix_time
0,Amsterdam,2013-01-01,14.0,1356969600000
1,Amsterdam,2013-01-02,8.0,1357056000000
2,Amsterdam,2013-01-03,12.0,1357142400000
3,Amsterdam,2013-01-04,12.0,1357228800000
4,Amsterdam,2013-01-05,14.0,1357315200000
...,...,...,...,...
156059,MARYSVILLE - 7TH AVE (Marysville Junior High),2023-03-30,7.9,1680105600000
156060,MARYSVILLE - 7TH AVE (Marysville Junior High),2023-03-31,3.7,1680192000000
156061,MARYSVILLE - 7TH AVE (Marysville Junior High),2023-04-01,3.4,1680278400000
156062,MARYSVILLE - 7TH AVE (Marysville Junior High),2023-04-02,3.1,1680364800000


Connecting to Hopsworks Feature Store

In [36]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Access the API key
HOPSWORKS_API_KEY = os.getenv('HOPSWORKS_API_KEY')
HOPSWORKS_PROJECT_NAME  = 'airqualityjith'

# Now you can use hopsworks_api_key in your code


In [37]:
import hopsworks
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/330902


In [38]:
fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.


In [40]:
from great_expectations.core import ExpectationSuite, ExpectationConfiguration

expectation_suite = ExpectationSuite(expectation_suite_name="pmi_data")

expectation_suite.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "pm2_5", 
            "min_value": 0.0,
            "max_value": 1000.0,
        }
    )
)

{"meta": {}, "kwargs": {"column": "pm2_5", "min_value": 0.0, "max_value": 1000.0}, "expectation_type": "expect_column_values_to_be_between"}

🪄 Creating Feature Groups
🌫 Air Quality Data

In [41]:
air_quality_fg = fs.get_or_create_feature_group(
    name='air_quality',
    description='Air Quality characteristics of each day',
    version=1,
    primary_key=['city_name'], #'unix_time',
    online_enabled=False,
    expectation_suite = expectation_suite,
    event_time="unix_time"
)    

In [42]:
air_quality_fg.insert(df_air_quality, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/330902/fs/329797/fg/352616
2023-12-26 03:04:39,485 INFO: 	1 expectation(s) included in expectation_suite.
Validation failed.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/330902/fs/329797/fg/352616


Uploading Dataframe: 100.00% |██████████| Rows 156064/156064 | Elapsed Time: 00:38 | Remaining Time: 00:00


Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/330902/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fc8682827d0>,
 {
   "meta": {
     "great_expectations_version": "0.14.13",
     "expectation_suite_name": "pmi_data",
     "run_id": {
       "run_time": "2023-12-25T19:04:39.484682+00:00",
       "run_name": null
     },
     "batch_kwargs": {
       "ge_batch_id": "7411a05c-a358-11ee-9b7f-f18e260080c0"
     },
     "batch_markers": {},
     "batch_parameters": {},
     "validation_time": "20231225T190439.484487Z",
     "expectation_suite_meta": {
       "great_expectations_version": "0.14.13"
     }
   },
   "success": false,
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 0,
     "unsuccessful_expectations": 1,
     "success_percent": 0.0
   },
   "results": [
     {
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2023-12-25T07:04:39.000484Z"
       },
       "result": {
         "element_count": 156064,
         "missing_count": 0,
         "missing_p

In [43]:
weather_fg = fs.get_or_create_feature_group(
    name='weather',
    description='Weather characteristics of each day',
    version=1,
    primary_key=['city_name'], #'unix_time'
    online_enabled=False,
    event_time="unix_time"
) 

In [44]:
weather_fg.insert(df_weather, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/330902/fs/329797/fg/353629


Uploading Dataframe: 100.00% |██████████| Rows 169515/169515 | Elapsed Time: 00:55 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/330902/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fc8691ad5d0>, None)